In [1]:
import geopandas as gpd
import numpy as np
import pandas as pd 
import pickle
import glob
import os
import pulp as plp #package used to solve Mixed Integer Programming Problem # https://pyomo-simplemodel.readthedoc
import time
from pathlib import Path
import config
import itertools
from multiprocessing import Pool
from itertools import repeat
import signal

In [2]:
def power_scaling(wpc_in, flh_const=4000):
    '''
    Function is used to scale the wind power capacity array to a value 
    in line with a maximum FLH constraint of flh_const

    Parameters
    ----------
    wpc_in : np.array
        Numpy array containing the Wind power capacity values per turbine and location.
    flh_const: int
        Integer indicating the Maxiumum FLH constraint used for defining the scaling factor

    Returns
    -------
    wpc_out : np.array
        Array containing the scaled Wind power capacity values.

    '''
    # Scale Full load hours by the maxiumum FLH of the smaller turbine is 4000:
    flh_max = np.max(wpc_in[:, 0]) * 8760/3
    if flh_max > 4000:
        scaling = 4000/flh_max
    else:
        scaling = 1
    
    wpc_out = wpc_in * scaling
    
    return wpc_out

def sound_pressure_level(sound_power, distances, hub_height):
    '''
    Function that calculates the sound pressure level at several distances 
    depending on sound_ower and hub_height
    '''
    return sound_power - abs(10* np.log10(1/(4*np.pi* np.square(distances))))

def round_down(num, divisor):
    '''
    Function for rounding down a number (num) depening on a divisor.
    (Eg. With divisor 10 the number is rounded down to the nearest 10: 26-->20)
    '''
    return num - (num%divisor)

def sound_pressure_level_uj(sound_power, distances, hub_height):
    '''
    Function calculating the sound pressure level as defined in the paper 
    by Jensen et al. (2014)
    '''
    return sound_power - 10*np.log10(np.square(distances) + np.square(hub_height)) - 11 + 1.5 - (2/1000) * np.sqrt((np.square(distances) + np.square(hub_height)))

In [3]:
def calc_power_cost(gdf, calc_type, study, ec=True):
    '''
    Function calculates relevant cost and power data for each cell in the dataset

    Parameters
    ----------
    gdf : geopandas.GeoDataFrame
        DataFrame containing all trubine placement locations and cell specific information
        (such as wind speed, capacity factor, impacted houses at 500m, ...).
    calc_type : string
        String defining the wind power calculation type used. Can be either wpf (based on
        the wind power function), cf (based on rated power and capacity factor) or pc (based
        on the power curve values given by the producer)
    study : string
        String used to define the externality cost assumption used for the cost calculation.
        dk: Droes & Koster (2016) and jensen: Jensen et al. (2014)
    ec : Bolean
        Bolean value which indicates if externalities should be used for the cost calculation
        (default=True).

    Returns
    -------
    total_cost : np.array
        Array containing the total cost (externality + installation cost) of each turbine
        in a specific placement location.
    wpc : np.array
        Array containing the wind power capacity values for each turbine and 
        placement location.
    external_cost : np.array
        Array containing the externality cost of each turbine 
        in a specific placement location.
    install_cost : np.array
        Array containing the installation cost of each turbine
        in a specific placement location.

    '''
    
    ws_100, ws_150 = gdf['ws_100'].to_numpy(), gdf['ws_150'].to_numpy()

    ws = np.array([ws_100, ws_150]).T
    
    if calc_type == "cf":
        cf_1, cf_t2 = gdf['iec1'].to_numpy(), gdf['iec2'].to_numpy()
        cf = np.array([cf_1, cf_t2]).T


    if calc_type == 'wpf':
        ad_100, ad_150 = gdf['ad_100'].to_numpy(), gdf['ad_150'].to_numpy()
        ad = np.array([ad_100, ad_150]).T
    sqm_p, ap_size, no_ap = gdf['sqm_p'].to_numpy(), gdf['ap_size'].to_numpy(), gdf['no_ap'].to_numpy()
    

    imp_list = [f"imp_{i}m" for i in list(range(500, 2750, 250))]

    impacts = gdf.loc[:, imp_list].to_numpy()


    hp = sqm_p * ap_size * no_ap # simple average sqm price 2077 per m^2 (replace for relevant calculation)
    

    impacts_hp = impacts * hp.reshape(-1, 1) # calculates total of house prices for the affected buildings in each zone

    if study == 'jensen':
        dmg = {0: 0.0, 10: 0.0, 20: 0.0307, 30: 0.055, 40: 0.0669, 50: 0.0669, 60: 0.0669}
        
        distance = np.array(range(500, 2750, 250)) - 125 # average distance of a cell within a certain distance range
        hub_height = np.array([[100], [150]])
        sound_power = np.array([[105.5], [106.1]]) #maximum sound power of E-115 and E-126EP3


        DB = round_down(sound_pressure_level_uj(sound_power, distance, hub_height), 10)

        
        cn = DB.copy()
        for noise_level in list(dmg.keys()):
            cn[cn == noise_level] = dmg[noise_level]

        cost_vis = {'500': (0.0315 + 22.5 * 0.0024),
                    '750': (0.0315 + 20.0 * 0.0024),
                    '1000': (0.0315 + 17.5 * 0.0024),
                    '1250': (0.0315 + 15.0 * 0.0024),
                    '1500': (0.0315 + 12.5 * 0.0024),
                    '1750': (0.0315 + 10.0 * 0.0024),
                    '2000': (0.0315 + 7.5 * 0.0024),
                    '2250': (0.0315 + 5.0 * 0.0024),
                    '2500': (0.0315 + 2.5 * 0.0024),
                    } #'2750': (0.0315 + 0.0 * 0.0024), '3000': (0.0315 + 0.0 * 0.0024)

        cv = np.array(list(cost_vis.values()))
        cost_ext = cn + cv
        
    elif study == 'dk':
        dk_dmg = {250: 0.026, 500: 0.026, 750: 0.025, 1000: 0.021, 1250: 0.019, 1500: 0.019, 1750: 0.015, 2000: 0.0, 2250: 0.0}
        cost_ext = np.array([list(dk_dmg.values()), list(dk_dmg.values())])


    install_cost = np.array([[(990 + 387 + 56 ) * 3000.0, 
                              (1180 + 387 + 56) * 4200.0]]) # from windguard study

    
    if study == "noext":
        external_cost = np.zeros((len(gdf), 2))
    else:
        external_cost = np.round(impacts_hp @ cost_ext.T, 2)
    
    total_cost = external_cost + install_cost

    # Calculate Wind power capacity:

    blade_radius = np.array([58, 63.5]).T # .reshape(1,2)

    # Calculate power coefficients:

    # Write down efficiency factor cp for the specific turbine at different wind speed (keys)
    enercon_e115_cp = {1: 0.0, 2: 0.058, 3: 0.227 , 4: 0.376, 5: 0.421, 6: 0.451 , 7: 0.469, 8: 0.470, 9: 0.445, 10: 0.401, 11: 0.338, 12: 0.270, 13: 0.212, 14: 0.170 , 15: 0.138,
                        16: 0.114, 17: 0.095, 18: 0.080, 19: 0.068, 20: 0.058, 21: 0.050, 22: 0.044, 23: 0.038, 24: 0.034, 25: 0.030}

    # nominal power 3000kW

    enercon_e126_cp = {1: 0.00, 2: 0.00, 3: 0.28, 4: 0.37, 5: 0.41 , 6: 0.44, 7: 0.45, 8: 0.45, 9: 0.43, 10: 0.40, 11: 0.35, 12: 0.30, 13: 0.24, 14: 0.20, 15: 0.16,
                     16: 0.13, 17: 0.11, 18: 0.09, 19: 0.08, 20: 0.07, 21: 0.06, 22: 0.05, 23: 0.04, 24: 0.04, 25: 0.03}

    # nominal power 4200kW

    enercon_e115_power = {1: 0.0, 2: 3.0, 3: 48.5 , 4: 155.0, 5: 339.0 , 6: 627.5 , 7: 1035.5, 8: 1549.0, 9: 2090.0, 10: 2580.0, 
                       11: 2900.0, 12: 3000.0, 13: 3000.0, 14: 3000.0, 15: 3000.0, 16: 3000.0, 17: 3000.0, 18: 3000.0, 19: 3000.0,
                        20: 3000.0, 21: 3000.0, 22: 3000.0, 23: 3000.0, 24: 3000.0, 25: 3000.0} 

    enercon_e126_power= {1: 0.0, 2: 0.0, 3: 58.0, 4: 185.0, 5: 400.0, 6: 745.0, 7: 1200.0, 8: 1790.0, 9: 2450.0 , 10: 3120.0, 
                         11: 3660.0, 12: 4000.0, 13: 4150.0 , 14: 4200.0 , 15: 4200.0 , 16: 4200.0, 17: 4200.0, 18: 4200.0, 
                         19: 4200.0, 20: 4200.0 , 21: 4200.0, 22: 4200.0, 23: 4200.0, 24: 4200.0 , 25:  4200.0}


    ##########################################################################################################################

    cp_100 = np.round(ws_100.copy())

    for wind_speed in list(enercon_e115_cp.keys()):
        cp_100[cp_100 == wind_speed] = enercon_e115_cp[wind_speed]

    cp_150 = np.round(ws_150.copy())

    for wind_speed in list(enercon_e126_cp.keys()):
        cp_150[cp_150 == wind_speed] = enercon_e126_cp[wind_speed]

    cp = np.vstack((cp_100, cp_150)).T

    ###########################################################################################################################
    wp_100 = np.round(ws_100.copy())

    for wind_speed in list(enercon_e115_power.keys()):
        wp_100[wp_100 == wind_speed] = enercon_e115_power[wind_speed]

    wp_150 = np.round(ws_150.copy())

    for wind_speed in list(enercon_e126_power.keys()):
        wp_150[wp_150 == wind_speed] = enercon_e126_power[wind_speed]

    wp = np.vstack((wp_100, wp_150)).T


    ###########################################################################################################################
    
    if calc_type == 'pc':
        # Caclulate wpc based on enercon power curve info
        wpc = wp * 0.001

    elif calc_type == 'wpf':
        # Calculate wpc based on wind speed and air density formula:
        wpc = (np.pi/2) * ws**3 * blade_radius**2 * ad * cp * 1e-6  # the 1e-6 are needed to convert from W to MW
    
    elif calc_type == "cf":
        rated = np.tile(np.array([3.0, 4.2]), (len(gdf), 1))     
        wpc = rated * cf
        
    else:
        print('Add valid calculation type')
    
        
    return total_cost, wpc , external_cost, install_cost

def custom_round(x, base=5):
    return int(base * round(float(x)/base))

In [4]:
WIPLEX_config = config.WIPLEX_settings()
WIPLEX_config.initialize_config()
param = WIPLEX_config.param
paths = WIPLEX_config.paths

In [5]:
df_types = pd.read_excel(paths["turbine_types_file"], sheet_name=0)
df_types["reference_height"] = df_types["hub_height"].apply(lambda x: custom_round(x, 50))

In [6]:
gdf_in = gpd.read_file("E:\Master_Thesis\WIPLEX\Database\Intermediate Outputs\current\opt_DEU_500.shp")

In [7]:
print(df_types.head())

  producer          name  rated_power  rotor_diameter  hub_height  tip_height  \
0  Enercon       E-70 E4         2.30           71.00          54      89.500   
1  Enercon  E-115 EP3 E3         2.99          115.70          92     149.850   
2  Enercon  E-138 EP3 E2         4.20          138.25         149     218.125   
3  Vestas    V172-7.2 MW         7.20          172.00         175     261.000   

   sound_power  cut_in_speed  cut_out_speed   wind_class  reference_height  
0        104.5           2.5             34       IEC IA                50  
1        104.8           2.5             34       IEC IA               100  
2        106.0           2.0             28     IEC IIIA               150  
3        106.9           3.0             25  DIBt (IIIA)               200  


In [31]:

dmg = {0: 0.0, 10: 0.0, 20: 0.0307, 30: 0.055, 40: 0.0669, 50: 0.0669, 60: 0.0669}

distance = np.array(range(500, 2750, 250)) - 125 # average distance of a cell within a certain distance range
hub_height = np.array([[i] for i in df_types["hub_height"]])
sound_power = np.array([[i] for i in df_types["sound_power"]])


DB = round_down(sound_pressure_level_uj(sound_power, distance, hub_height), 10)


cn = DB.copy()
for noise_level in list(dmg.keys()):
    cn[cn == noise_level] = dmg[noise_level]

cost_vis = {'500': (0.0315 + 22.5 * 0.0024),
            '750': (0.0315 + 20.0 * 0.0024),
            '1000': (0.0315 + 17.5 * 0.0024),
            '1250': (0.0315 + 15.0 * 0.0024),
            '1500': (0.0315 + 12.5 * 0.0024),
            '1750': (0.0315 + 10.0 * 0.0024),
            '2000': (0.0315 + 7.5 * 0.0024),
            '2250': (0.0315 + 5.0 * 0.0024),
            '2500': (0.0315 + 2.5 * 0.0024),
            } #'2750': (0.0315 + 0.0 * 0.0024), '3000': (0.0315 + 0.0 * 0.0024)

cv = np.array(list(cost_vis.values()))
cost_ext = cn + cv

In [32]:
# TODO: Implement Windguard cost analysis:

install_cost = np.array([[(990 + 387 + 56 ) * 3000.0, 
                          (1180 + 387 + 56) * 4200.0]]) # from windguard study

[[40. 30. 30. 30. 20. 20. 20. 20. 20.]
 [40. 30. 30. 30. 20. 20. 20. 20. 20.]
 [40. 30. 30. 30. 30. 20. 20. 20. 20.]
 [40. 30. 30. 30. 30. 20. 20. 20. 20.]]
[[0.1524 0.1345 0.1285 0.1225 0.0922 0.0862 0.0802 0.0742 0.0682]
 [0.1524 0.1345 0.1285 0.1225 0.0922 0.0862 0.0802 0.0742 0.0682]
 [0.1524 0.1345 0.1285 0.1225 0.1165 0.0862 0.0802 0.0742 0.0682]
 [0.1524 0.1345 0.1285 0.1225 0.1165 0.0862 0.0802 0.0742 0.0682]]


# Test SOME STUFF BELOW

In [6]:
pa_gdf = gpd.read_file(f'{paths["gwa_placement_area"]}', crs=param["epsg_general"])

In [25]:
pa_gdf.columns

Index(['ad_100', 'ad_150', 'ad_200', 'ad_50', 'iec1', 'iec2', 'iec3', 'ws_100',
       'ws_150', 'ws_200', 'ws_50', 'geometry', 'imp_500m', 'imp_750m',
       'imp_1000m', 'imp_1250m', 'imp_1500m', 'imp_1750m', 'imp_2000m',
       'imp_2250m', 'imp_2500m', 'imp_2750m', 'imp_3000m', 'imp_3250m',
       'imp_3500m', 'imp_3750m', 'imp_4000m', 'imp_4250m', 'imp_4500m',
       'imp_4750m', 'imp_5000m', 'imp_5250m', 'imp_5500m', 'imp_5750m',
       'imp_6000m'],
      dtype='object')

In [26]:
def buffer_geoms(gdf_in, buffer, proj_epsg=25832):
    '''
    Function calculates a buffered GeoSeries from the geometry column of a geopandas DataFrame
    in a given meter based coordinate reference system (crs) and converts it back to the orginial crs
    '''
    init_crs = gdf_in.crs
    gdf_in = gdf_in.to_crs(proj_epsg)
    buffered = gpd.GeoDataFrame(geometry = gdf_in.geometry.buffer(buffer)).to_crs(init_crs)

    return buffered

def add_house_impact_allocation(param_dict, paths_dict):
    '''
    Function calculates for a given set of impact zones (in meter) the number of affected houses per impact zone for each
    cell in a given raster.

    Parameters
    ----------
        param_dict : dict
            parameter dictionary specified in the config.py file
        paths_dict : dict
            paths dictionary specified in the config.py file

    Returns
    -------
        None
    '''
    osmfiles = glob.glob(os.path.join(paths_dict["osm_path"], '*.zip'))
    filename = paths_dict["osm_file_names"]["buildings"]
    #sub_files = np.array_split(osmfiles, 6)

    # # Read in shapefile with house prices:
    # hp_gdf = gpd.read_file(f'{paths_dict["house_prices"]}', crs=param_dict["epsg_general"])
    # hp_gdf.loc[:, 'state'].replace({'-':'_', 'ü': 'ue', 'ä': 'ae', 'ö': 'oe'}, regex=True, inplace=True)
    # hp_gdf.rename(columns={'avg_house_': 'hp', 'sqm_euro_v':'sqm_p', 'avg_apartm':'ap_size','no_appartm':'no_ap'}, inplace=True)

    # Calculate number of affected houses per impact zone for each raster cell
    print('Started calculating impact allocation')
    start = time.process_time() # start time check for calculations
    pa_gdf = gpd.read_file(f'{paths_dict["gwa_placement_area"]}', crs=param_dict["epsg_general"])
    col_len = len(pa_gdf.columns)

    for buff in param_dict["impact_buffers"]:
        print(f'Calculating {buff}m impacts...')
        pa_buff = buffer_geoms(pa_gdf, buff , param_dict["epsg_distance_calc"])

        inter_list = []
        for z in osmfiles:
            # osm_list = []
            # for file in z:
            #print(f"Reading buildings data from {z}/{filename}")
            osm_gdf = gpd.read_file(f"zip://{z}/{filename}").loc[:,['geometry']].to_crs(param_dict["epsg_distance_calc"])
            osm_gdf = osm_gdf.set_geometry(osm_gdf.centroid).to_crs(param_dict["epsg_general"])
            #osm_list.append(osm_gdf)

            # Only consider residential areas
            gdf_res = gpd.read_file(f"zip://{z}/{paths_dict['osm_file_names']['landuse']}").loc[:, ['fclass','geometry']]
            gdf_res = gdf_res.loc[(gdf_res['fclass'] == 'residential')]
            osm_gdf = gpd.clip(osm_gdf, gdf_res)

            #reg_gdf = pd.concat(osm_list)
            #dfsjoin = gpd.sjoin(pa_buff, reg_gdf, how="left") #Spatial join Points to polygons
            dfsjoin = gpd.sjoin(pa_buff, osm_gdf, how="left") #Spatial join Points to polygons
            counts = dfsjoin.groupby(dfsjoin.index)["index_right"].count().to_numpy()

            inter_list.append(counts) # add np.array of intersection count for file z to a list
        

        inter_vector = np.sum(inter_list, axis=0) # combine array list first to a single array and then sum over the relevant array column
        pa_gdf[f'imp_{buff}m'] = inter_vector # Add impact to output GeoDataFrame
        print(f"Finished {buff}m impact zone. Time passed:", time.process_time() - start)

    # Correct impact double-counting:
    # To avoid double counting of impacts subtract the larger buffer zone by the next smaller one
    # The smallest buffer zone impact count will simply be subtracted by 0 
    impacts_dc = pa_gdf.iloc[:, col_len:].copy().to_numpy()
    sub_array= np.hstack((np.zeros((impacts_dc.shape[0],1)), impacts_dc[:, :-1])) # adds zero column to numpy array in position 0
    corrected_impacts = impacts_dc - sub_array
    pa_gdf.iloc[:, col_len:] = corrected_impacts.astype(int)

    # # Add house price information to each cell
    # print('Adding house price information')
    # joined_gdf = gpd.sjoin(pa_gdf, hp_gdf, how="inner", op="intersects")
    # joined_gdf.drop(columns=['index_right'], inplace=True)
    # joined_gdf = joined_gdf[~joined_gdf.index.duplicated(keep="first")] # drop duplicates keep first

    # Save file:
    # joined_gdf.to_file(f'{paths_dict["optimization_file"]}', index=True)
    pa_gdf.to_file(f'{paths_dict["optimization_file"]}', index=True)
    print('Finished generating impact allocation')

    return None

def add_house_price_info(param_dict, paths_dict):
    """
    Function adds house price information to optimization GeoDataFrame (ogdf) based on an inner spatial
    join with the ogdf rows (Expl.: Information is added to row if geometry of the ogdf is inside of the house price
    region of the house price gdf)

    Parameters
    ----------
        param_dict : dict
            parameter dictionary specified in the config.py file
        paths_dict : dict
            paths dictionary specified in the config.py file
    """
    # Read in shapefile with house prices:
    hp_gdf = gpd.read_file(f'{paths_dict["house_prices"]}', crs=param_dict["epsg_general"])
    hp_gdf.loc[:, 'state'].replace({'-':'_', 'ü': 'ue', 'ä': 'ae', 'ö': 'oe'}, regex=True, inplace=True)
    hp_gdf.rename(columns={'avg_house_': 'hp', 'sqm_euro_v':'sqm_p', 'avg_apartm':'ap_size','no_appartm':'no_ap'}, inplace=True)

    # Read in shapefile with turbine locations:
    pa_gdf = gpd.read_file(f'{paths_dict["optimization_file"]}', crs=param_dict["epsg_general"])

    # Add house price information to each raster cell
    print('Adding house price information')
    joined_gdf = gpd.sjoin(pa_gdf, hp_gdf, how="inner", op="intersects")
    joined_gdf.drop(columns=['index_right'], inplace=True)
    joined_gdf = joined_gdf[~joined_gdf.index.duplicated(keep="first")] # drop duplicates keep first

    # Save file:
    joined_gdf.to_file(f'{paths_dict["optimization_file"]}', index=True)
    print("Final Number of cells:", len(joined_gdf))

In [8]:
for buff in param["impact_buffers"]:
    pa_gdf[f'imp_{buff}m'] = 3000 # Add impact to output GeoDataFrame

In [23]:
param["impact_buffers"] = [500]

In [24]:
add_house_impact_allocation(param, paths)

Started calculating impact allocation
Calculating 500m impacts...
Finished 500m impact zone. Time passed: 2065.953125
Finished generating impact allocation


In [27]:
add_house_price_info(param, paths)

Adding house price information
Final Number of cells: 518702


In [10]:
pa_gdf.to_file(f'test.shp', index=True)


In [28]:
res = gpd.read_file(f'{paths["optimization_file"]}', crs=param["epsg_general"])

In [29]:
res.columns

Index(['level_0', 'index', 'ad_100', 'ad_150', 'ad_200', 'ad_50', 'iec1',
       'iec2', 'iec3', 'ws_100', 'ws_150', 'ws_200', 'ws_50', 'imp_500m',
       'county', 'CC_2', 'pop_count', 'pop_d_km2', 'state', 'sqm_p', 'ap_size',
       'no_ap', 'hp', 'geometry'],
      dtype='object')

In [30]:
import fiona

In [31]:
fiona.supported_drivers  

{'AeronavFAA': 'r',
 'ARCGEN': 'r',
 'BNA': 'raw',
 'DXF': 'raw',
 'CSV': 'raw',
 'OpenFileGDB': 'r',
 'ESRIJSON': 'r',
 'ESRI Shapefile': 'raw',
 'GeoJSON': 'rw',
 'GeoJSONSeq': 'rw',
 'GPKG': 'rw',
 'GML': 'raw',
 'GPX': 'raw',
 'GPSTrackMaker': 'raw',
 'Idrisi': 'r',
 'MapInfo File': 'raw',
 'DGN': 'raw',
 'PCIDSK': 'r',
 'S57': 'r',
 'SEGY': 'r',
 'SUA': 'r',
 'TopoJSON': 'r'}